In [2]:
from caveclient import CAVEclient
from meshparty import trimesh_io, trimesh_vtk
import cloudvolume 
import numpy as np


In [3]:
client = CAVEclient('minnie65_public_v117')


In [4]:
# to access dynamic meshes, you can query the segmentation source from the info client
client.info.segmentation_source()

'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public_v117'

In [11]:
# this can be used to initialize a cloudvolume object
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)
# which given a root_id can be used to get a mesh
# cloud volume returns a dictionary with meshes as the keys
mesh = cv.mesh.get(864691135474648896)[864691135474648896]


In [12]:
# meshes are triangular meshes with vertices and faces
# vertices are Nx3 x,y,z positions in nm
# faces are Kx3 i,j,k indices into vertices that describe triangles
mesh.vertices.shape, mesh.faces.shape


((1698627, 3), (3388543, 3))

In [14]:
# since downloading meshes can take some time, particularly for these dynamic meshes
# it is convient to cache them on disk.
# also there a number of analytical techniques that you often want to accomplish on meshes.
# such as constructed a graph of the vertices and edges and calculating distances on them.
# to facilitate this we developed a package called MeshParty

# to enable a cache, create a MeshMeta object
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_v117_meshes',
                         map_gs_to_https=True)
# now i can get a mesh throught this and it will be cached in memory
# and in disk in case i need it again.
# restart the kernel and run all the cells again to see the difference.
# you'll find the mesh file saved as an hdf5 file in the "minnie65_v117_meshes"
# subdirectory
mesh = mm.mesh(seg_id=864691135474648896)

The MeshParty object has more useful properties and attributes
such as a scipy.csgraph sparse graph object (mesh.csgraph) and a networkx 
graph object (mesh.nxgraph) 

Read more about what you can do with MeshParty on its [Documentation](https://meshparty.readthedocs.io/en/latest/?badge=latest).

In particular it lets you associate skeletons and annotations onto the mesh into a "meshwork" object.  

The meshes that are available in the visualizaton on micronsexplorer are fast because they are static and all located in a single file, and because they were downsampled to 3 different levels of downsampling.  This makes them faster to interactive and more useful for many applications.  They also do not have a sense of the morphology of the neuron and so gaps 